In [1]:
import json
import betfairlightweight
import queue
import pandas as pd

In [2]:
# Import libraries
import os
import betfairlightweight
from copy import deepcopy
import pandas as pd
import queue
import datetime
from datetime import datetime as dt
from IPython.display import Image
import json
import time

In [3]:
with open('secrets/config.json', 'r') as fp:
    config = json.load(fp)

In [29]:
trading = betfairlightweight.APIClient(**config)
trading.login()

/Users/jake/OneDrive/Projects/sides/bf_hft/secrets


<LoginResource>

In [112]:
mrkt = '1.166505727'
SIZE = 0.03

In [108]:
def process_runner_books(runner_books):
    '''
    This function processes the runner books and returns a DataFrame with the best back/lay prices + vol for each runner
    :param runner_books:
    :return:
    '''
    best_back_prices = [runner_book.ex.available_to_back[0].price
                        if runner_book.ex.available_to_back
                        else 1.01
                        for runner_book
                        in runner_books]
    best_back_sizes = [runner_book.ex.available_to_back[0].size
                       if runner_book.ex.available_to_back
                       else 1.01
                       for runner_book
                       in runner_books]

    best_lay_prices = [runner_book.ex.available_to_lay[0].price
                       if runner_book.ex.available_to_lay
                       else 1000.0
                       for runner_book
                       in runner_books]
    best_lay_sizes = [runner_book.ex.available_to_lay[0].size
                      if runner_book.ex.available_to_lay
                      else 1.01
                      for runner_book
                      in runner_books]
    
    selection_ids = [runner_book.selection_id for runner_book in runner_books]
    last_prices_traded = [runner_book.last_price_traded for runner_book in runner_books]
    total_matched = [runner_book.total_matched for runner_book in runner_books]
    statuses = [runner_book.status for runner_book in runner_books]
    scratching_datetimes = [runner_book.removal_date for runner_book in runner_books]
    adjustment_factors = [runner_book.adjustment_factor for runner_book in runner_books]

    df = pd.DataFrame({
        'Selection ID': selection_ids,
        'Best Back Price': best_back_prices,
        'Best Back Size': best_back_sizes,
        'Best Lay Price': best_lay_prices,
        'Best Lay Size': best_lay_sizes,
        'Last Price Traded': last_prices_traded,
        'Total Matched': total_matched,
        'Status': statuses,
        'Removal Date': scratching_datetimes,
        'Adjustment Factor': adjustment_factors
    })
    return df

In [111]:
# Create a price filter. Get all traded and offer data
price_filter = betfairlightweight.filters.price_projection(
    price_data=['EX_BEST_OFFERS']
)

# Request market books
market_books = trading.betting.list_market_book(
    market_ids=[mrkt],
    price_projection=price_filter
)

# Grab the first market book from the returned list as we only requested one market 
market_book = market_books[0]

runners_df = process_runner_books(market_book.runners)

runners_df.sort_values('Total Matched')



,Selection ID,Best Back Price,Best Back Size,Best Lay Price,Best Lay Size,Last Price Traded,Total Matched,Status,Removal Date,Adjustment Factor
6,3840518,50.00,7.96,70.0,59.01,55.0,237.80,ACTIVE,NaT,1.625
2,4639452,42.00,22.80,44.0,62.65,40.0,383.48,ACTIVE,NaT,3.546
5,27281179,22.00,10.40,23.0,10.27,20.0,686.21,ACTIVE,NaT,5.850
0,6580083,22.00,26.83,23.0,26.38,22.0,871.01,ACTIVE,NaT,6.803
8,27281180,15.00,16.49,16.5,14.84,16.0,1198.00,ACTIVE,NaT,3.441
3,13184837,19.50,159.79,21.0,24.15,19.5,1444.94,ACTIVE,NaT,3.441
10,25645745,15.00,14.41,16.5,65.47,16.5,1921.51,ACTIVE,NaT,4.500
1,27281178,6.60,409.02,6.8,200.17,6.6,5226.34,ACTIVE,NaT,22.757
9,20862458,7.00,82.55,7.4,303.45,7.4,5504.49,ACTIVE,NaT,11.471
4,17118120,5.60,12.51,5.7,60.16,5.7,6564.96,ACTIVE,NaT,18.283


In [113]:
sid = runners_df[runners_df['Total Matched'] == runners_df['Total Matched'].max()]['Selection ID'].values[0]
price = runners_df[runners_df['Total Matched'] == runners_df['Total Matched'].max()]['Best Back Price'].values[0]

In [114]:
sid, price

(25088281, 4.3)

In [118]:
# Define a limit order filter
limit_order_filter = betfairlightweight.filters.limit_order(
    size=SIZE, 
    price=price,
    persistence_type='LAPSE'
)

# Define an instructions filter
instructions_filter = betfairlightweight.filters.place_instruction(
    selection_id=str(sid),
    order_type="LIMIT",
    side="BACK",
    limit_order=limit_order_filter
)

instructions_filter



{'limitOrder': {'size': 0.03, 'persistenceType': 'LAPSE', 'price': 1.01},
 'side': 'BACK',
 'selectionId': '25088281',
 'orderType': 'LIMIT'}

In [119]:
# Place the order
order = trading.betting.place_orders(
    market_id=mrkt, # The market id we obtained from before
    customer_strategy_ref='scratch_pad',
    instructions=[instructions_filter] # This must be a list
)

In [120]:
order.__dict__

{'elapsed_time': 1.756299,
 '_datetime_created': datetime.datetime(2019, 12, 21, 6, 32, 23, 57562),
 '_datetime_updated': datetime.datetime(2019, 12, 21, 6, 32, 23, 57562),
 '_data': {'status': 'SUCCESS',
  'marketId': '1.166505727',
  'instructionReports': [{'status': 'SUCCESS',
    'instruction': {'selectionId': 25088281,
     'limitOrder': {'size': 0.03, 'price': 1.01, 'persistenceType': 'LAPSE'},
     'orderType': 'LIMIT',
     'side': 'BACK'},
    'betId': '189347653435',
    'placedDate': '2019-12-21T06:32:22.000Z',
    'averagePriceMatched': 4.2,
    'sizeMatched': 0.03,
    'orderStatus': 'EXECUTION_COMPLETE'}]},
 'market_id': '1.166505727',
 'status': 'SUCCESS',
 'customer_ref': None,
 'error_code': None,
 'place_instruction_reports': [<betfairlightweight.resources.bettingresources.PlaceOrderInstructionReports at 0x1239250f0>]}

In [83]:
p_and_l = trading.betting.list_market_profit_and_loss(market_ids=[mrkt])

In [84]:
p_and_l[0].__dict__

{'elapsed_time': 2.344491,
 '_datetime_created': datetime.datetime(2019, 12, 21, 6, 14, 49, 624781),
 '_datetime_updated': datetime.datetime(2019, 12, 21, 6, 14, 49, 624781),
 '_data': {'marketId': '1.166570560',
  'profitAndLosses': [{'selectionId': 11832228, 'ifWin': 0.0},
   {'selectionId': 10416431, 'ifWin': 0.0},
   {'selectionId': 16555023, 'ifWin': 0.0},
   {'selectionId': 22968361, 'ifWin': 0.0},
   {'selectionId': 22152037, 'ifWin': 0.0},
   {'selectionId': 2057206, 'ifWin': 0.0},
   {'selectionId': 27290329, 'ifWin': 0.0},
   {'selectionId': 10416647, 'ifWin': 0.0},
   {'selectionId': 24350233, 'ifWin': 0.0},
   {'selectionId': 27290330, 'ifWin': 0.0},
   {'selectionId': 4322460, 'ifWin': 0.0},
   {'selectionId': 16637986, 'ifWin': 0.0}]},
 'market_id': '1.166570560',
 'commission_applied': None,
 'profit_and_losses': [<betfairlightweight.resources.bettingresources.ProfitAndLosses at 0x123917588>,
  <betfairlightweight.resources.bettingresources.ProfitAndLosses at 0x123917470

In [79]:
cancelled_order = trading.betting.cancel_orders(market_id=mrkt)

In [80]:
cancelled_order.__dict__

{'elapsed_time': 2.01735,
 '_datetime_created': datetime.datetime(2019, 12, 21, 6, 11, 8, 703340),
 '_datetime_updated': datetime.datetime(2019, 12, 21, 6, 11, 8, 703340),
 '_data': {'status': 'SUCCESS',
  'marketId': '1.166570560',
  'instructionReports': [{'status': 'SUCCESS',
    'instruction': {'betId': '189346304706'},
    'sizeCancelled': 0.03,
    'cancelledDate': '2019-12-21T06:11:08.000Z'}]},
 'market_id': '1.166570560',
 'status': 'SUCCESS',
 'customer_ref': None,
 'error_code': None,
 'cancel_instruction_reports': [<betfairlightweight.resources.bettingresources.CancelOrderInstructionReports at 0x123917748>]}

In [30]:
current_orders = trading.betting.list_current_orders()

In [31]:
df = pd.DataFrame([current_orders.orders[x].__dict__ for x in range(len(current_orders.orders))])

In [33]:
df

""


In [ ]:
agg_df = df[['average_price_matched', 'market_id', 'side']].groupby(
    ['market_id', 'side'], as_index=False).agg(
    {'average_price_matched': ['count', 'mean']})

agg_df

In [18]:
agg_df.columns = ['market_id', 'side', 'count', 'avg']

In [26]:
back_price = agg_df[agg_df.side=='BACK'].sort_values('market_id')['avg'].values
lay_price = agg_df[agg_df.side=='LAY'].sort_values('market_id')['avg'].values
counts = agg_df[['market_id', 'count']].groupby('market_id').mean()['count'].values

In [27]:
(back_price - lay_price) * counts

array([0.02])

In [55]:
[x.customer_strategy_ref for x in current_orders.orders]

['scratch_pad', 'scratch_pad']

In [56]:
tp_orders = set()

In [68]:
if not tp_orders:
    print(tp_orders)

set()


In [65]:
tp_orders.add('thing')

In [66]:
tp_orders.remove('thing')

In [34]:
account_statement = trading.account.get_account_statement()

In [37]:
pd.DataFrame(account_statement.__dict__['_data']['accountStatement'])

,refId,itemDate,amount,balance,itemClassData,legacyData,itemClass
0,189498473344,2019-12-22T10:13:16.000Z,0.03,9.97,"{'unknownStatementItem': '{""avgPrice"":2.0,""bet...","{'avgPrice': 2.0, 'betSize': 0.03, 'betType': ...",UNKNOWN
1,189498463244,2019-12-22T10:13:16.000Z,-0.05,9.94,"{'unknownStatementItem': '{""avgPrice"":2.5,""bet...","{'avgPrice': 2.5, 'betSize': 0.03, 'betType': ...",UNKNOWN
2,189498455309,2019-12-22T10:13:16.000Z,-0.05,9.99,"{'unknownStatementItem': '{""avgPrice"":2.52,""be...","{'avgPrice': 2.52, 'betSize': 0.03, 'betType':...",UNKNOWN
3,189498433387,2019-12-22T10:13:16.000Z,-0.05,10.04,"{'unknownStatementItem': '{""avgPrice"":2.82,""be...","{'avgPrice': 2.82, 'betSize': 0.03, 'betType':...",UNKNOWN
4,189498423895,2019-12-22T10:13:16.000Z,-0.05,10.09,"{'unknownStatementItem': '{""avgPrice"":2.68,""be...","{'avgPrice': 2.68, 'betSize': 0.03, 'betType':...",UNKNOWN
...,...,...,...,...,...,...,...
95,188470476343,2019-12-14T15:57:56.000Z,-41.32,2277.38,"{'unknownStatementItem': '{""avgPrice"":2.12,""be...","{'avgPrice': 2.12, 'betSize': 36.89, 'betType'...",UNKNOWN
96,188054705046,2019-12-14T14:23:39.000Z,-0.91,2318.70,"{'unknownStatementItem': '{""avgPrice"":0.0,""bet...","{'avgPrice': 0.0, 'betSize': 0.0, 'betType': '...",UNKNOWN
97,188054705046,2019-12-14T14:23:39.000Z,18.24,2319.61,"{'unknownStatementItem': '{""avgPrice"":16.5,""be...","{'avgPrice': 16.5, 'betSize': 18.24, 'betType'...",UNKNOWN
98,188461218721,2019-12-14T14:00:17.000Z,-12.07,2301.37,"{'unknownStatementItem': '{""avgPrice"":0.0,""bet...","{'avgPrice': 0.0, 'betSize': 0.0, 'betType': '...",UNKNOWN


In [38]:
cleared_orders = trading.betting.list_cleared_orders(bet_status="SETTLED")


In [39]:
df = pd.DataFrame(cleared_orders._data['clearedOrders'])
df = df[df.eventTypeId=='7']

In [40]:
df.sort_values('customerStrategyRef')

,eventTypeId,eventId,marketId,selectionId,handicap,betId,placedDate,persistenceType,orderType,side,betOutcome,priceRequested,settledDate,lastMatchedDate,betCount,priceMatched,priceReduced,sizeSettled,profit,customerStrategyRef
74,7,29626789,1.166505723,15568681,0.0,189343597028,2019-12-21T05:17:24.000Z,LAPSE,LIMIT,BACK,LOST,5.60,2019-12-21T05:22:17.000Z,2019-12-21T05:17:24.000Z,1,5.60,False,0.03,-0.03,1.166505723_bot
46,7,29626789,1.166505725,23807494,0.0,189345631337,2019-12-21T05:56:14.000Z,LAPSE,LIMIT,LAY,WON,3.10,2019-12-21T06:03:42.000Z,2019-12-21T05:56:14.000Z,1,3.10,False,0.03,0.03,1.166505725_bot
40,7,29626789,1.166505725,23807494,0.0,189345332239,2019-12-21T05:50:23.000Z,LAPSE,LIMIT,LAY,WON,2.92,2019-12-21T06:03:42.000Z,2019-12-21T05:50:23.000Z,1,2.92,False,0.03,0.03,1.166505725_bot
43,7,29626789,1.166505725,23807494,0.0,189345351589,2019-12-21T05:50:44.000Z,LAPSE,LIMIT,LAY,WON,2.88,2019-12-21T06:03:42.000Z,2019-12-21T05:50:44.000Z,1,2.86,False,0.03,0.03,1.166505725_bot
45,7,29626789,1.166505725,23807494,0.0,189345589078,2019-12-21T05:55:30.000Z,LAPSE,LIMIT,LAY,WON,2.96,2019-12-21T06:03:42.000Z,2019-12-21T05:55:31.000Z,1,2.96,False,0.03,0.03,1.166505725_bot
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49,7,29626789,1.166505725,23807494,0.0,189345683190,2019-12-21T05:57:17.000Z,LAPSE,LIMIT,LAY,WON,2.94,2019-12-21T06:03:42.000Z,2019-12-21T05:57:17.000Z,1,2.94,False,0.03,0.03,NaN
71,7,29626836,1.166505811,24619265,0.0,189344845840,2019-12-21T05:40:21.000Z,LAPSE,LIMIT,BACK,LOST,7.40,2019-12-21T05:42:22.000Z,2019-12-21T05:40:21.000Z,1,7.40,False,0.03,-0.03,NaN
72,7,29626836,1.166505811,20185019,0.0,189344845841,2019-12-21T05:40:21.000Z,LAPSE,LIMIT,BACK,WON,1.88,2019-12-21T05:42:22.000Z,2019-12-21T05:40:21.000Z,1,1.88,False,0.02,0.02,NaN
73,7,29626836,1.166505811,14683384,0.0,189344845842,2019-12-21T05:40:21.000Z,LAPSE,LIMIT,LAY,WON,40.00,2019-12-21T05:42:22.000Z,2019-12-21T05:40:21.000Z,1,40.00,False,0.02,0.02,NaN


In [43]:
df[['marketId', 'side', 'profit']].groupby(['marketId', 'side'], as_index=False).sum()

,marketId,side,profit
0,1.166505723,BACK,-0.03
1,1.166505723,LAY,0.03
2,1.166505725,BACK,-0.15
3,1.166505725,LAY,0.15
4,1.166505727,BACK,0.10
5,1.166505727,LAY,-0.09
6,1.166505803,BACK,-0.03
7,1.166505803,LAY,0.03
8,1.166505809,BACK,-0.03
9,1.166505809,LAY,0.03


In [44]:
from utils import *

In [51]:
def _tick_delta(price, delta):
    """Computes the price given a price and a tick difference"""
    ticks = dollars_to_ticks(price)
    ticks += delta
    ret_price = ticks_to_dollars(ticks)
    return round(ret_price, 2)

In [49]:
dollars_to_ticks(1.88) - dollars_to_ticks(1.90)

-2

In [52]:
_tick_delta(1.88, 2)

1.89

In [55]:
dollars_to_ticks(1.88)

87

In [57]:
(1.88 - 1) / 0.01

87.99999999999999

In [58]:
int(round((1.88 - 1) / 0.01))

88

In [59]:
round(1.89)

2